In [40]:
import sys
sys.path.append('..')
import math
import numpy as np
import lumapi
from src.Polarization.extract_polar import extrac_polar_lr
from src.Polarization.set_angle import set_angle

path = 'D:\\polarization_control\\Square2D_new'
fdtd = lumapi.FDTD(filename = path)
 
kx = fdtd.getnamed('FDTD', 'kx') # in the bandstructure unit
ky = fdtd.getnamed('FDTD', 'ky')
f = 563.52e12 # in Hz unit
a = 380e-9 # dimension m
k_unit = 2 * math.pi / a
omega = 2 * math.pi * f
k_p = k_unit * np.array([kx, ky])

polar_1 = extrac_polar(fdtd, 'monitor_xy_1', omega, k_p)
polar_2 = extrac_polar(fdtd, 'monitor_xy_2', omega, k_p)
fdtd.close()
#T = np.linalg.norm(np.abs(polar))**2 / 2
#print(T)

Data to compare the transmission of Lumerical's and self-calculated

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
angle = np.array([-60, -32, -4, 12, 32, 60])
T_fdtd = [-0.717063, -0.805799, -0.603131, -0.793976, -0.805799, -0.717063 ]
T_self = [-0.6882345716441882, -0.7892865933661235, -0.6012820556195664, -0.6012820644146154, -0.7908602963961807, -0.7890992886179404,0.6882345769117709]

The following part is used to test the correctness of calculated phase and polarization, and calculate the phase vortex

In [1]:
import sys
sys.path.append('..')
import math
import numpy as np
import lumapi
import matplotlib.pyplot as plt
from src.Polarization.extract_polar import extrac_polar
from src.Polarization.set_angle import set_angle
%load_ext autoreload
%autoreload 2

path = 'D:\\polarization_control\\Square2D_new'
fdtd = lumapi.FDTD(filename = path)

f = 563.52e12 # in Hz unit
a = 380e-9 # dimension m
k_unit = 2 * math.pi / a
omega = 2 * math.pi * f

num = 36
theta = 4 * np.ones(num) # in degree
phi = np.linspace(0, 360, num) # in degree
coeff_1, coeff_2 = [], []
trans_1, trans_2 = [], []

for theta_i, phi_i in zip(theta, phi):
    kp = set_angle(fdtd, 'source_x', theta_i, phi_i) # in bandstructure unit
    set_angle(fdtd, 'source_y', theta_i, phi_i)
    k_p = k_unit * kp
    fdtd.run()
    polar_1 = extrac_polar_lr(fdtd, 'monitor_xy_1', omega, k_p)
    #polar_2 = extrac_polar(fdtd, 'monitor_xy_2', omega, k_p)
    T_1 = np.linalg.norm(np.abs(polar_1))**2 / 2
    #T_2 = np.linalg.norm(np.abs(polar_1))**2 / 2

    trans_1.append(T_1)
    coeff_1.append(polar_1)
    #trans_2.append(T_2)
    #coeff_2.append(polar_2)
    sys.stdout.write('.')
print('')
fdtd.close()



ModuleNotFoundError: No module named 'Lumerical_methods'

In [29]:
%matplotlib widget
coeff_o = np.array(np.load('coeff_532nm_xy_nostruct.npy'))
coeff = np.array(coeff_1)
phase = np.angle(coeff_o[:, 1], deg = True)
plt.plot(phi, phase)
plt.title('Figure of Phase')
plt.xlabel('$\phi$ (degree)')
plt.ylabel('Phase of $E_(ave)$(degree)')
plt.show()

FFT analysis of the fields

In [20]:
import sys
sys.path.append('..')
sys.path.append('../src/')
import math
import numpy as np
import lumapi
from src.Polarization.extract_polar import extrac_polar
from src.Polarization.set_angle import set_angle
from src.Lumerical_methods.lumerical_scripts import get_field

path = 'D:\\polarization_control\\Square2D_new'
fdtd = lumapi.FDTD(filename = path)

kx = fdtd.getnamed('FDTD', 'kx') # in the bandstructure unit
ky = fdtd.getnamed('FDTD', 'ky')
f = 563.52e12 # in Hz unit
a = 380e-9 # dimension m
k_unit = 2 * math.pi / a
omega = 2 * math.pi * f
k_p = k_unit * np.array([kx, ky])

fields = get_field(fdtd, 'monitor_xy_1', field_result_name = 'fields', get_eps = False, get_D = False, get_H = False, noninterpolation = False)
E_fields = fields.E[:, :, 0, 0, 0]
fdtd.close()

xx, yy = np.meshgrid(fields.x, fields.y)
kernel = np.exp(-1j *(k_p[0] * xx + k_p[1] * yy))
E_shifted = np.transpose(E_fields) * kernel
E_fft = np.fft.fft2(E_shifted)


KeyboardInterrupt: 

In [15]:
import matplotlib.pyplot as plt
%matplotlib widget
fft_shift = np.fft.fftshift(E_fft)
plt.imshow(np.abs(fft_shift), )
plt.colorbar()
plt.title('k-space amplitude')
plt.gca().invert_yaxis()

In [21]:
np.save('coeff_532nm_xy_nostruct',coeff)

In [16]:
import numpy as np
a = np.array([[1],[2],[3]])
b = np.array([[0, 8, 9], [0,5,6], [0,2,3]])
(b.sum(1)).shape

(3,)

In [25]:
import sys
import numpy as np
sys.path.append('..')
from src.Polarization.extract_polar import s_p_coeff
F_ave = np.array([[1,1,3], [3,1,2], [2,1,3]])
omega = 3e9
k_p = np.array([[1,0,1], [1,1,0]])
s_p_coeff(F_ave=F_ave, omega=omega, k_p=k_p)

array([[1, 1, 1],
       [1, 3, 2]])